In [1]:
import xarray as xr
from netCDF4 import Dataset, date2num
from datetime import datetime, timedelta

In [2]:
with xr.open_mfdataset('cpc/*nc') as dset:
    print(dset)

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 3653)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 2011-01-01 2011-01-02 ... 2020-12-31
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(365, 360, 720), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.0
    version:        V1.0
    history:        created 9/2016 by CAS NOAA/ESRL PSD
    title:          CPC GLOBAL PRCP V1.0 RT
    dataset_title:  CPC GLOBAL PRCP V1.0
    Source:         ftp://ftp.cpc.ncep.noaa.gov/precip/CPC_UNI_PRCP/
    References:     https://www.psl.noaa.gov/data/gridded/data.cpc.globalprec...


### Correção das coordenadas ...

In [3]:
dset = dset.reindex(lat=dset['lat'][::-1])

/home/funceme/.local/lib/python3.7/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [4]:
# dset.lat.values

In [5]:
dset['lon_aux'] = xr.where(
    dset['lon'] > 180,
    dset['lon'] - 360,
    dset['lon']
)

dset = (
    dset
        .swap_dims({'lon': 'lon_aux'})
        .sel(**{'lon_aux': sorted(dset.lon_aux)})
        .drop('lon')
)

dset = dset.rename({'lon_aux': 'lon'})

/home/funceme/.local/lib/python3.7/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [6]:
# dset.lon.values

In [7]:
# Seleciona região e os anos 2019 e 2020
pr_mon = dset['precip'].sel(
    lat=slice(-9.5, -1),
    lon=slice(-44, -35),
    time=(slice('2019', '2020')))

pr_mon

<xarray.DataArray 'precip' (time: 731, lat: 17, lon: 18)>
dask.array<getitem, shape=(731, 17, 18), dtype=float32, chunksize=(366, 17, 18), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -9.25 -8.75 -8.25 -7.75 ... -2.75 -2.25 -1.75 -1.25
  * time     (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2020-12-31
  * lon      (lon) float32 -43.75 -43.25 -42.75 -42.25 ... -36.25 -35.75 -35.25
Attributes:
    var_desc:      Precipitation
    level_desc:    Surface
    statistic:     Total
    parent_stat:   Other
    long_name:     Daily total of precipitation
    cell_methods:  time: sum
    avg_period:    0000-00-01 00:00:00
    actual_range:  [  0.      465.00006]
    units:         mm
    valid_range:   [   0. 1000.]
    dataset:       CPC Global Precipitation

In [8]:
pr_mon.shape

(731, 17, 18)

In [9]:
# https://unidata.github.io/netcdf4-python/

fname = '/home/funceme/my_notebooks/precip.cpc.daily.total.2019-2020.nc'  # Nome do arquivo NetCDF

ncfile = Dataset(fname, 'w', format='NETCDF4_CLASSIC')  # Abre o NetCDF para escrita

# Cria as dimensões:

ncfile.createDimension('time', None)
ncfile.createDimension('lat', 17)        # num pontos latitude
ncfile.createDimension('lon', 18)        # num pontos longitude

# Cria as variáveis:

# float = 'f4', double = 'f8'

# Eixo usados: time, lat, lon

lats = ncfile.createVariable('lat', 'f4', ('lat'))
lats.units = 'degrees_north'
lats.long_name = 'latitude'               # Nome descritivo do eixo lat 
lats.axis = 'Y'
lats[:] = pr_mon['lat'].values            # Numpy Array

lons = ncfile.createVariable('lon', 'f4', ('lon'))
lons.units = 'degrees_east'
lons.long_name = 'longitude'              # Nome descritivo do eixo lon
lons.axis = 'X'
lons[:] = pr_mon['lon'].values            # Numpy Array

# Data Inicial: 2019,1,1
# Num de passos de tempo: 731
list_dates = []                           # Lista com objetos datetime
for day in range(731):
    list_dates.append(datetime(2019, 1, 1) + timedelta(days=day))

# or list comprehension:
# list_dates = [datetime(2019, 1, 1) + timedelta(days=day) for day in range(731)]
    
times = ncfile.createVariable('time', 'f4', ('time'))
times.units = 'days since 2019-01-01 00:00:00'
times.axis = 'T'
times.calendar = 'standard'
times.long_name = 'time'
times[:] = date2num(list_dates, times.units)
# times[:] = date2num(list_dates, units='days since 2019-01-01 00:00:00', calendar='standard')

var = ncfile.createVariable(
    'precip',                             # Nome da variável
    'f4',
    ('time', 'lat', 'lon'),
    zlib=True                             # Compressão no NetCDF
)
var.units = 'mm'                          # Unidade da variável
var.long_name = 'Precipitation'           # Nome descritivo da variável
var[:] = pr_mon.values                    # Numpy Array

ncfile.source = 'CPC NOAA PSL'            # Fonte dos dados

ncfile.description = 'CPC Monthly Total'  # Descrição dos dados

ncfile.comment = 'Try to recreate CPC'    # Comentários

today = datetime.today()
ncfile.history = 'Created: ' + today.strftime("%b %d, %Y")

ncfile.close()                            # Fecha a escrita do NetCDF

In [10]:
list_dates

[datetime.datetime(2019, 1, 1, 0, 0),
 datetime.datetime(2019, 1, 2, 0, 0),
 datetime.datetime(2019, 1, 3, 0, 0),
 datetime.datetime(2019, 1, 4, 0, 0),
 datetime.datetime(2019, 1, 5, 0, 0),
 datetime.datetime(2019, 1, 6, 0, 0),
 datetime.datetime(2019, 1, 7, 0, 0),
 datetime.datetime(2019, 1, 8, 0, 0),
 datetime.datetime(2019, 1, 9, 0, 0),
 datetime.datetime(2019, 1, 10, 0, 0),
 datetime.datetime(2019, 1, 11, 0, 0),
 datetime.datetime(2019, 1, 12, 0, 0),
 datetime.datetime(2019, 1, 13, 0, 0),
 datetime.datetime(2019, 1, 14, 0, 0),
 datetime.datetime(2019, 1, 15, 0, 0),
 datetime.datetime(2019, 1, 16, 0, 0),
 datetime.datetime(2019, 1, 17, 0, 0),
 datetime.datetime(2019, 1, 18, 0, 0),
 datetime.datetime(2019, 1, 19, 0, 0),
 datetime.datetime(2019, 1, 20, 0, 0),
 datetime.datetime(2019, 1, 21, 0, 0),
 datetime.datetime(2019, 1, 22, 0, 0),
 datetime.datetime(2019, 1, 23, 0, 0),
 datetime.datetime(2019, 1, 24, 0, 0),
 datetime.datetime(2019, 1, 25, 0, 0),
 datetime.datetime(2019, 1, 26, 0,

### Abrir NetCDF no Ferret para verificar ...